### BW43 - 커스텀 컨테이너 타입은 collections.abc를 상속하자.

- 파이썬 프로그래밍의 핵심은 데이터를 담은 클래스를 정의하고 이 객체들이 연계되는 방법을 명시하는 일이다. <br/> 모든 파이썬 클래스는 일종의 컨테이너고 속성과 기능을 함께 캡슐화한다. <br/><br/>
- 시퀀스처럼 사용법이 간단한 클래스를 정의할 때는 <br/> 파이썬 내장 리스트 타입의 하위 클래스를 만들고 싶은것이 당연합니다. <br/> Q. 왜 당연할까요? 에? <br/> A. 속성들을 이용할수 있어서이다.

Ex. 멤버의 빈도를 계산하는 메서드를 추가로 갖춘 커스텀 list 타입을 생성
<br/> A. 리스의 하위 클래스를 만들어준다. 단순히 추가 목저이니까..

In [5]:
class FrequencyList(list):         # 리스트를 상속밭아 리스트 본연의 기능을 수행됨.
    def __init__(self, members):   # 빈도를 세는 프리퀀시라는 메서드가 추가됨.
        super().__init__(members)
                                    
    def frequency(self):           # 리스트에서 상속받아 서브클래스를 만들어줌.
        counts = {}
        for item in self:
            counts[item] = counts.get(item, 0) + 1
        return counts

foo = FrequencyList(['a', 'b', 'a', 'c', 'b', 'a', 'd'])
print('길이: ', len(foo))

foo.pop()
print('pop한 다음:', repr(foo))
print('빈도:', foo.frequency())

길이:  7
pop한 다음: ['a', 'b', 'a', 'c', 'b', 'a']
빈도: {'a': 3, 'b': 2, 'c': 1}


- 파이썬 프로그래머들이라면 이런 함수들의 의미가 뭔지 알것이다. <br/> 추가로 필요한 기능을 제공하는 메서드를 얼마든지 추가할 수 있다고 합니다.

<hr/>

~~list 상속만해도 많은 작업을 무난하게 시행 할것 같은데 굳이 더 복잡한걸 예시로 할려고 한다.~~ <br/><br/>
Ex. list 처럼 느껴지면서 인덱싱이 가능한 객체를 제공 받고 싶은데
<br/> 단, 리스트의 하위 클래스로 만들고 싶지 않다고 가정해보면 ~~왜 굳이~~
<br/><br/> Ex1. 이진 트리 클래스를 시퀀스의 의미 구조를 사용해 다룰 수 있는 클래스를 만들고 싶다.

In [6]:
class BinaryNode:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

Q. 클래스가 시퀀스 처럼 동작하게 하라면 어떻게 해야 할까?
<br/> A. 파이썬에서는 특별한 이름의 인스턴스 메서드를 사용해 컨테이너의 동작을 구현한다.

In [7]:
bar = [1, 2, 3] # 리스트의 첫 원소를 구하는 예제다.
bar[0]

1

In [8]:
bar.__getitem__(0) # 객체의 트리를 깊이 우선순회하는 __getitem__을 구현 하면 된다.

1

> 깊이우선순회 ?
- 루트 노드에서 시작해서 다음 분기로 넘어가기 전에 해당 분기를 완벽하게 탐색하는 방법
- 자기 자신 호출 순환 알고리즘

In [9]:
class IndexableNode(BinaryNode):
    def _traverse(self):
        if self.left is not None:
            yield from self.left._traverse()
        yield self
        if self.right is not None:
            yield from self.right._traverse()

    def __getitem__(self, index):
        for i, item in enumerate(self._traverse()):
            if i == index:
                return item.value
        raise IndexError(f'인덱스 범위 초과: {index}')


이진 트리를 만든다.

In [10]:
tree = IndexableNode(
    10,
    left=IndexableNode(
        5,
        left=IndexableNode(2),
        right=IndexableNode(
            6,
            right=IndexableNode(7))),
    right=IndexableNode(
        15,
        left=IndexableNode(11)))

- 트리는 시퀀스와 달리 비선형 자료구조이지만 탐색포함, list처럼 접근할 수도 있다.

In [11]:
print('LRR:', tree.left.right.right.value)
print('인덱스 0:', tree[0])
print('인덱스 1:', tree[1])
print('11이 트리 안에 있나?', 11 in tree)
print('17이 트리 안에 있나?', 17 in tree)
print('트리:', list(tree))

LRR: 7
인덱스 0: 2
인덱스 1: 5
11이 트리 안에 있나? True
17이 트리 안에 있나? False
트리: [2, 5, 6, 7, 10, 11, 15]


- 문제는 __ getitem__을 구현 하는 것만으로는 리스트 인스턴스에서 기대할 수 있는 모든 시퀀스 의미 구조를 제공할 없다.

<hr/>

Ex2. 내장 함수 len을 쓸려면 커스텀 시퀀스 타입에 맞게 구현한 __len__ 이라는 또 다른 특별한 메서드가 필요.

In [12]:
len(tree) # 메서드 또한 시퀀스에서 사용자가 시행할 수 있는 작업이다.

TypeError: object of type 'IndexableNode' has no len()

In [13]:
class SequenceNode(IndexableNode):
    def __len__(self):
        for count, _ in enumerate(self._traverse(), 1):
            pass
        return count

In [15]:
tree = SequenceNode(
    10,
    left=SequenceNode(
        5,
        left=SequenceNode(2),
        right=SequenceNode(
            6,
            right=SequenceNode(7))),
    right=SequenceNode(
        15,
        left=SequenceNode(11))
)

In [16]:
print('트리 길이:', len(tree))

트리 길이: 7


- 특정 메소드를 정의해주면 시퀀스 의미 구조를 지원할 수 있다
<br/> 하지만 역시나 문제가 있다 ... <br/> 시퀀스 의미 구조에서 사용자가 기대하는 메서드 인덱싱이 다가 아니기 때문이다. <br/> count, index, 등과 같은 메서드가 더 필요하다. <br/><br/> Q. 메서드들을 다 정의 해줘야 할까?
<br/> A. 생각보다 커스텀 컨테이너 타입을 직접 정의하는 건 보기보다 어렵다.

<hr/>

- 내장 collections.abc 모듈을 상속하자 <br/><br/> 
- 모듈안에는 시퀀스 구조에 요구되는 필수 메서드를 구현하지 않은 채 정의만하고 있다. <br/> 정의하는 클래스가 시퀀스 구조를 모두 지원하겠다는 선언을 하는 것이다.
- 따라서 구현이 안되어 있으면 충족되지 않았다고, 실수했다고 알려줍니다.

In [17]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

foo = BadType()

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

In [18]:
class BetterNode(SequenceNode, Sequence):
    pass

tree = BetterNode(
    10,
    left=BetterNode(
        5,
        left=BetterNode(2),
        right=BetterNode(
            6,
            right=BetterNode(7))),
    right=BetterNode(
        15,
        left=BetterNode(11))
)

print('7의 인덱스:', tree.index(7))
print('10의 개수:', tree.count(10))

7의 인덱스: 3
10의 개수: 1


- 파이썬의 관례에 맞춰 구현해야 하는 특별한 메서드가 많은 더 복잡한 타입을 정의할 때 
<br/> 이런 추상 기반 클래스를 사용하는 이점은 더욱 커진다.
<br/> ~~Q1. 타입을 직접 정의하는 것은 생각보다 훨씬 더 어려운 일이라고 예시..~~
<br/> Q2. 요구하는 메서드를 거저 얻는 다고 써있는데 병렬적으로 메서드를 추가하는게 아닐것 같아서 쉬운게 아닐것 같은데 어떻게 생각하시나요?

### 기억하자
- 간편하게 사용할 경우에는 파이썬 컨테이너 타입(리스트나 딕셔너리 등)을 직접 상속하자.
- 커스텀 컨테이너를 제대로 구현하려면 수많은 메서드를 구현해야 한다는 점에 주의하자.
- 커스텀 컨테이너 타입이 collection.abc에 정의된 인터페이스를 상속하면 <br/> 커스텀 컨테이너 타입이 정상적으로 작동하기 위해 필요한 인터페이스와 기능을 제대로 구현하도록 보장할 수 있다.